In [34]:
import numpy as np # import for math and array operations
import pandas as pd # import for dataframe handle
import matplotlib.pyplot as plt # import for visual representation
import seaborn as sns # import for visual representation
import math
from bs4 import BeautifulSoup
import requests
import datetime as dt

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
df = pd.read_csv('../raw_data/steam_games.csv')

# Cleaning columns

### URL

### Types

In [6]:
new_df = df.copy().dropna(subset=['types'])
app_df = new_df[new_df['types'] == 'app']

### Names

### Desc_snippet

In [10]:
# Complete desc_snippet with game_description
for index in app_df['desc_snippet'].index:
    app_df.loc[index, 'desc_snippet'] = app_df.loc[index, 'game_description']

### Reviews

In [11]:
def clean_review(review):
    if 'score' in str(review):
        return review.split(',')[0]
    else:
        return float('nan')

### Dates

In [19]:
def convert_date(date):
    try:
        return dt.datetime.strptime(date, "%b %d, %Y")
    except:
        return date
    
def valid_date(date):
    if isinstance(date, dt.datetime):
        return True
    else:
        return False

In [20]:
app_df['release_date'] = app_df['release_date'].apply(convert_date)

/var/folders/dv/7t_6tkbx07s6mfq_r5jxkp240000gp/T/ipykernel_28408/3659942068.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app_df['release_date'] = app_df['release_date'].apply(convert_date)


In [30]:
# add a validate column
app_df['valid_date'] = app_df['release_date'].apply(valid_date)

/var/folders/dv/7t_6tkbx07s6mfq_r5jxkp240000gp/T/ipykernel_28408/1590340263.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app_df['valid_date'] = app_df['release_date'].apply(valid_date)


### Developer

### Publisher

In [ ]:
### Publisher cleaning sub
def clean_pub(x):
    if ',' in str(x):
        if x.split(',')[0] == x.split(',')[1]:
            return x.split(',')[0] 
df['publisher'] = df['publisher'].apply(lambda x : clean_pub(x))

### Popular tags

In [27]:
### creating a loop of all the different tags, calculating the occurences to select the top used.
tags = {}
for index, row in df.iterrows():
    tags_list= str(row['popular_tags']).split(',')
    for tag in tags_list:
        if not tag in tags:
            tags[tag] = 1
        else:
            tags[tag] += 1
            
            
tags_df = pd.DataFrame(list(tags.items()),columns = ['tag','count'])
tags_df.sort_values('count', ascending =False)

,tag,count
50,Indie,23718
2,Action,16112
42,Adventure,13378
76,Casual,12969
31,Simulation,9273
...,...,...
362,ATV,3
369,Snowboarding,3
351,Snow,3
203,Asymmetric VR,3


### Languages

In [26]:
### creating a loop of all the different languages, calculating the occurences to select the top used.
languages = {}
for index, row in df.iterrows():
    languages_list= str(row['languages']).split(',')
    for language in languages_list:
        if not language in languages:
            languages[language] = 1
        else:
            languages[language] += 1

languages_df = pd.DataFrame(list(languages.items()),columns = ['language','count'])
languages_df.sort_values('count', ascending =False)

,language,count
0,English,40718
3,German,13053
1,French,12515
4,Spanish - Spain,11555
8,Russian,9724
...,...,...
59,#lang_#lang_spanish*#lang_full_audio,1
60,Slovakian,1
61,(all with full audio support),1
63,#lang_german;,1


### Game details

In [32]:
### creating a loop of all the different details, calculating the occurences to select the top used.
details = {}
# Stripping game details
df.game_details = df.game_details.str.strip()
for index, row in df.iterrows():
    details_list= str(row['game_details']).split(',')
    for detail in details_list:
        if not detail in details:
            details[detail] = 1
        else:
            details[detail] += 1

details_df = pd.DataFrame(list(details.items()),columns = ['game_details','count'])
details_df.sort_values('count', ascending =False)

,game_details,count
0,Single-player,36463
3,Steam Achievements,22045
14,Downloadable Content,14815
4,Steam Trading Cards,13219
6,Steam Cloud,13155
15,Full controller support,10012
1,Multi-player,9096
25,Profile Features Limited,8443
5,Partial Controller Support,7748
18,Steam Leaderboards,6801


### Achievements

In [36]:
#categorization - adding a column for achievement cat
for i in df.index:
    df.at[i, 'achievement_cat'] = 0 if math.isnan(df.loc[i]['achievements']) == True else 1

# Scraping Columns

### Names

In [37]:
def get_name(url):
    response = requests.get(url).text
    soup = BeautifulSoup(response, "html.parser")
    try: 
        return soup.find('h2', class_='pageheader').text.strip()
    except AttributeError:
        try:
            return soup.find('div', class_='apphub_AppName').text.strip()
        except AttributeError:
            return float('nan')

In [9]:
for index in app_df[app_df['name'].isnull()].index:
    app_df.loc[index, 'name'] = get_name(app_df.loc[index, 'url'])

NameError: name 'get_name' is not defined

### Reviews

In [ ]:
def get_review(url):
    response = requests.get(url).text
    soup = BeautifulSoup(response, "html.parser")
    try: 
        return soup.find('span', class_='game_review_summary').text.strip()
    except AttributeError:
        return float('nan')

In [ ]:
for index in app_df[app_df['all_reviews'].isnull()].index:
    app_df.loc[index, 'all_reviews'] = get_review(app_df.loc[index, 'url'])

### Dates

In [12]:
def get_date(url):
    response = requests.get(url).text
    soup = BeautifulSoup(response, "html.parser")
    try: 
        return soup.find('div', class_='date').text.strip()
    except AttributeError:
        return float('nan')

In [ ]:
for index in app_df[app_df['release_date'].isnull()].index:
    app_df.loc[index, 'release_date'] = get_date(app_df.loc[index, 'url'])

### Developer

In [ ]:
def get_dev(url):
    response = requests.get(url).text
    soup = BeautifulSoup(response, "html.parser")
    try: 
        return soup.find('div', id='developers_list').text.strip().split(',')[0]
    except:
        return float('nan')

In [ ]:
temp = df_app[app_df['developer'].isnull() == True]
for index in temp.index:
    app_df.loc[index, 'developer'] = get_dev(app_df.loc[index, 'url'])
    print(index)

### Publisher

In [38]:
def get_pub(url):
    response = requests.get(url).text
    soup = BeautifulSoup(response, "html.parser")
    try: 
        return soup.find_all('div', class_='summary column')[3].find('a').string.strip().split(',')[0]
    except:
        return float('nan')

In [ ]:
temp = df_app[df_app['publisher'].isnull() == True]
for index in temp.index:
    df_app.loc[index, 'publisher'] = get_pub(df_app.loc[index, 'url'])

### Popular Tags

In [23]:
def get_tags(url):
    response = requests.get(url).text
    soup = BeautifulSoup(response, "html.parser")
    try: 
        return ','.join([tag.text.strip() for tag in soup.find_all('a', class_='app_tag')])
    except:
        return float('nan')

In [ ]:
temp = app_df[app_df['popular_tags'].isnull() == True]
for index in temp.index:
    app_df.loc[index, 'popular_tags'] = get_tags(app_df.loc[index, 'url'])
app_df['popular_tags'] = app_df['popular_tags'].replace('', float('nan'))

### Game Details

In [39]:
def get_det(url):
    response = requests.get(url).text
    soup = BeautifulSoup(response, "html.parser")
    try: 
        return ','.join([tag.text.strip() for tag in soup.find_all('div', class_='label')])
       # return soup.find_all('div', class_='label')
    except:
        return float('nan')

In [ ]:
temp = df_app[df_app['game_details'].isnull() == True]
for index in temp.index:
    app_df.loc[index, 'game_details'] = get_tags(app_df.loc[index, 'url'])
app_df['game_details'] = app_df['game_details'].replace('', float('nan'))

# Encoding columns

### Publisher

In [4]:
#### Replacing missing publisher by developer - add inplace = True
#df.publisher = df.publisher.fillna(df.developer)